In [1]:
def InvNewton(F,N):
    """
    F is in a PolynomialRing over any ring, and F(0)!=0, N>0 is a natural number.
    Returns the power series development of 1/F up to order N with Newton iteration (i.e. coeffs of 1,x,...,x^(N-1)).
    """
    
    A = F.parent()
    x= A.gen()
    
    if F%x==0:
        return 'Constant coefficient of the polynomial must be nonzero in order to invert in a power series'

    # We will develop 1/F up to x^(2^n)
    n = ceil(log(max(N,1),2))

    # Base case
    if n==0:
        return 1/F(0)

    # Recursive call
    G=InvNewton(F,2^(n-1))
    
    return G + (1-((F*G)%x^N))*G % x^N


def InvLaurent(F,N):
    """
    F is in a PolynomialRing over any ring and F(0) is invertible, N>=0 is a natural number.
    Returns a couple (v,G) where v is the order of the pole at 0 of 1/F, and G/x^v is the Laurent series development of 1/F up to order N (i.e. coeffs of x^(-v),...,x^(-1),1,x,...,x^(N-1)).
    Thus, G is a polyonmial of degree N+v-1.
    """
    
    A = F.parent()

    # Base case
    if F.degree() == 0:
        return (0,A(1/F(0)))
    x= A.gen()

    # We will develop up to x^(2^N)
    n=ceil(log(max(N,2),2))

    v = F.valuation()
    L = F.list()
    G = A(L[v:]) # The v first coefficient L[0],...,L[v-1] are 0 and L[v]!=0
    
    return (v,InvNewton(G,N+v))


def Taylor(P,Q,N):
    """
    P,Q are in the same PolynomialRing over any ring, Q is nonzero, N is a natural number.
    Returns the power series expansion of P/Q up to order N using Newton iteration function InvLaurent above.
    Laurent series are treated as shifted power series.
    """
    
    A=Q.parent()
    x=A.gen()
    d=Q.degree()
    if d==0:
        return A(P/Q)

    E,R=P.quo_rem(Q) # E is the entire, polynomial part of the quotient, P/Q = E + R/Q
    
    v,G = InvLaurent(Q,N)
    T=R*G/x^v # T=R/Q mod x^N
    
    return (A(x^v*(E + T))%x^(N+v))/x^v


def pcurvature(a,b,p):
    """
    a and b are two polynomials with integer coeffs, p a prime number
    Computes the power series expansion of the 1/p-th power of the p-curvature of y'=(a/b)y
    up to the degree 2*deg(b), and checks if it is zero
    This is an implementation of an algorithm designed by Bostan and Schost in a 2009 article.
    """

    A = b.parent()
    x = A.gen()
    F.<x> = GF(p)[]

    # Reduce mod p and remove common factors between a mod p and b mod p
    ap = F(a)
    bp = F(b)
    dp = gcd(ap,bp)
    ap = ap//dp
    bp = bp//dp

    if ap==0 or bp==0:
        return 0 # If bp == 0 the p-curvature is not necessarily zero, in practice p-curvature is not called if this happens.
    
    n=bp.degree()

    u = Taylor(ap,bp,2*n) # u = ap/bp mod p, mod x^(2n), this is a Laurent series
    if u==0:
        return 0
    
    v1 = u.valuation(x)
    shift_u = F((x^(-v1))*u).list() # u shifted to be a power series
    lu = len(shift_u) # lu = 2*n-v1 except if there are zeros coefficients u_{2*n-1},...
    # shift_u[0] = u_{v1}, ...

    s=0 # will be (b/a) differentiated p-1 times, to the power 1/p
    v2 = (v1//p)
    r2 = v1%p
    # s = -\sum_{i\geq 0} u_{ip+v0}x^{i+v2}, we will compute first x^(-v2)*s
    # the valuation of s is v2; the first nonzero term is u_{v0} with v0=v2*p+p-1
    # first potentially nonzero term of s: u_{v0}*x^{v2}

    # Computation of u_{ip-1} with Fiduccia's algo
    f=bp.reverse() # Characteristic polynomial of the coefficients of u
    B.<xi> = F.quotient_ring(f)

    # Computation of the first coeff u_{v2*p+p-1}*x^(v2), corresponding to shift_u[v2*p+p-1-v1], remark that v2*p-v1+p-1 = -r2+p-1 >= 0
    X0=xi^(p-1-r2)
    list_X0=X0.list()
    l = len(list_X0)
    for j in range(min(l,lu)): # If list_X0 is incomplete (0's at the end, i.e. if bp has degree <n) we ignore it by stopping at len(list_X0)
        s += -list_X0[j]*shift_u[j] # The minus sign from (p-1)! (Wilson's Theorem)

    # Computation of all other coeffs of x^(-v2)*s
    step = (xi^(r2+1))*X0 # step = xi^p
    for i in range(2,2*n+1):
        X0*=step #X0=xi^(ip-1-r2)
        list_X0=X0.list()
        l = len(list_X0)
        si=0 # Next coefficient of s
        for j in range(min(l,lu)): 
            si+=list_X0[j]*shift_u[j]
        s += -si*x^(i-1)
    s=s*x^(v2) # Unshift s
    return u+s


def Timepcurvs(a,b,S):
    """
    a and b are two polynomials with integer coeffs, S>0.
    Computes all p-curvatures of y' = (a/b)y for primes p<= S.
    Doesn't take into account special primes.
    This function can be used to estimate the time it takes to compute p-curvatures.
    """
    p=2

    # p-curvatures computation
    while p<=S:
        v = pcurvature(a,b,p)
        p=next_prime(p)


def Listpcurvs(a,b,S):
    """
    a and b are two polynomials with integer coeffs, S>0.
    Computes all p-curvatures of y' = (a/b)y for primes p<= S, and returns the list of all couples (p,v) where p is a prime and v a nonzero p-curvature.
    Doesn't take into account special primes.
    """
    p=2
    pcurvs=[]

    # p-curvatures computation
    while p<=S:
        v = pcurvature(a,b,p)
        if v!=0:
            pcurvs+=[(p,v)]
        p=next_prime(p)
    return pcurvs


def CompFactor(a,b):
    """
    a,b are two polynomials with integer coefficients.
    Computes the Rothstein-Trager resultant and factors it, then checks if all factors have degree 1.
    Returns algebraic or transcendental, the nature of solutions of y' = (a/b) y
    """
    A = b.parent()
    x = A.gen()
    if b.degree() <2:
        return 'algebraic'

    # Compute the Rothstein-Trager resultant
    G.<w>=QQ[]
    H.<x>=G[]
    t=H(b)
    u=H(a)
    R=t.resultant(u-w*t.derivative())

    # Factor R
    L=list(R.factor())

    # Checks the degree of the factors
    for c in L:
        if c[0].degree()!=1:
            return 'transcendental'
    return 'algebraic'


def RTres(P,Q):
    """
    P,Q are in a PolynomialRing A over any ring A0.
    Computes the Rothstein-Trager resultant of P and Q both seen in A = PolynomialRing(A0,'x').
    Returns the resultant in x of Q and P-w*(Q.derivative(x)), in PolynomialRing(A0,'w').
    """
    A = P.parent()
    x = A.gen()

    B0.<w> = (A.base_ring())[]
    B.<x> = B0[]

    P1 = B(P)
    Q1 = B(Q)

    return Q1.resultant(P1-w*Q1.derivative(x))


def dval(Delta):
    """
    Delta is a nonzero integer.
    Returns a numerical approximation of delta = prod_{p | Delta} p^{1/(p-1)} using integer factorization of Delta.
    """
    temp = 0
    for p in Delta.prime_factors():
        temp += (log(p)/(p-1)).n()
    return exp(temp)


# Possible value of C0 such that lcm(1,...,N) <= C0^N for all N >= 1
C0=(955888052326228459513511038256280353796626534577600^(1/113)+2^(-52)).n()


def max_root(f):
    """
    f is a polynomial with coefficients in ZZ, QQ, RR, or CC.
    Returns an upper bound on the maximal modulus of its roots in the complex field.
    Calls SageMath function f.roots(ComplexField) that calls a PARI implementation of a root-finding algorithm using real arithmetic.
    """
    m = 0

    Roots_f = f.roots(ComplexField()) # Computes a numerical approximation of the roots of f in CC with 53 bits of precision

    # For each root, we add a slight epsilon = +- 2^(-52) to the real and imaginary parts to ensure we obtain an upper bound on the moduli.
    for t in Roots_f:
        r = t[0]
        epsr = r.real_part().sign()*2^(-52) 
        epsi = r.imag_part().sign()*2^(-52)
        val = abs(r+(epsr+I*epsi)) # close upper bound on the modulus of the root
        if val>m:
            m=val

    return m


def sigma(a,b):
    """
    a,b are two polynomials with integer coefficients.
    Returns the bound sigma on the number of p-curvatures to check in order to ensure algebracity of solutions of y' = (a/b) y
    according to Theorem 1.3.
    """
    R = RTres(a,b) # Rothstein-Trager resultant
    Delta = abs(ZZ(R.leading_coefficient())) # Leading coefficient
    delta = dval(Delta) # The function dval factors Delta, careful for large integers
    
    B = max(ceil(max_root(R)),1) # Upper bound on the moduli of the complex roots of R

    M = ceil(C0*(Delta^3)*(delta^3))
    N = ceil(6.1*B*M)
    
    # print(f'Delta={Delta}, t={t}, B={B}, M={M}, N={N}')
    return (2*M+1)*N+2*M


def Honda(a,b):
    """
    a,b are polynomials with integer coefficients.
    Returns a string and a value p. The string is the nature, algebraic or transcendental of the solutions of y'=(b/a)y.
    If it is algebraic, then p is zero.
    Else the value p is the prime for which the non-vanishing of the p-curvature allowed to conclude on the transcendence of solutions.
    If p is 0 and the nature is transcendental then the algorithm concluded from the degree of the rational function or the presence of a double pole, this is specified in the string.
    
    The bound sigma on the primes to check is not computed totally to avoid costly computations that might be useless (Rothstein-Trager computation, factor(Delta),...)
    Primes up to Delta are tested naively at the beginning so that Delta is only factored step by step throughout the algorithm, the RT resultant is only computed afterwards if there was no conclusion yet.
    """
    
    x = b.parent().gen()
    A.<x> = ZZ[]

    # Reduction of a and b to content 1
    u=A(a)
    cu=u.content_ideal().gen(i=0)
    u=u/cu
    
    v=A(b)
    cv=v.content_ideal().gen(i=0)
    v=v/cv
    v0=v.leading_coefficient()

    # Checking the degree condition, i.e. if the equation is regular at infinity
    n=v.degree()
    if u.degree()>=n:
        return 'transcendental: degree of the rational function', 0

    # Computation of Delta
    Delta = abs(ZZ(v.resultant(v.derivative())))
    
    # Checking the pole condition, i.e. if the equation is regular at all finite points
    if Delta == 0:
        return 'transcendental: double pole', 0

    temp = 0 # This will become log(delta)
    
    # First p-curvatures computations
    p = 2
    while p<=Delta:
        if Delta%p == 0: # if yes, the prime is special
            temp += (log(p)/(p-1)).n() # Construct delta from those primes
        else: # We compute p-curvature and, if zero, we conclude immediately 
            C = pcurvature(u,v,p)
            if C != 0:
                return 'transcendental', p
        p = next_prime(p)
    # At this point p is greater than Delta, hence greater than all special primes
    # We have to compute sigma
    
    R = RTres(u,v)
    B = max(ceil(max_root(R)),1).n()
    delta = exp(temp).n() # delta is complete

    M = ceil(C0*(Delta^3)*(delta^3))
    N = ceil((6.1*B*M).n())
    S = (2*M+1)*N+2*M # This is our sigma
    
    # print(f'sigma = {S}')

    # Computation of all primes
    while p<=S:
        C = pcurvature(u,v,p)
        if C!=0:
            return 'transcendental', p
        p=next_prime(p)
    
    return 'algebraic', 0